In [24]:
import os 
from dotenv import load_dotenv
import pandas as pd
import requests

from binance import Client

In [25]:
load_dotenv()

True

In [26]:
BINANCE_API_KEY=os.getenv('BINANCE_API_KEY')
BINANCE_SECRET_KEY=os.getenv('BINANCE_SECRET_KEY')
client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY, tld='us')


In [27]:
client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY, tld='us')

In [28]:
trades = client.get_historical_trades(symbol='SOLUSD')

klines = client.get_historical_klines("SOLUSD", Client.KLINE_INTERVAL_1DAY, "30 Apr, 2020", "30 Apr, 2022")

In [29]:
pd.DataFrame(klines) 
klines_df = pd.DataFrame(klines) 

In [30]:

klines_df.columns = ['open time', 'open', 'high', 'low', 'close', 'volume', 'close time', 'quote asset volume', 'number of trades', 'taker buy base asset volume', 'taker buy quote asset volume', 'NaN']

In [31]:
klines_df.drop(columns='NaN')


,open time,open,high,low,close,volume,close time,quote asset volume,number of trades,taker buy base asset volume,taker buy quote asset volume
0,1600387200000,3.0888,3.2926,2.8050,3.1002,41039.60000000,1600473599999,124683.8036,714,26548.92000000,81833.3694
1,1600473600000,3.1002,3.2616,2.9846,3.0076,19297.28000000,1600559999999,59744.3755,686,7902.23000000,24579.4758
2,1600560000000,3.0042,3.0042,2.7550,2.8737,21274.03000000,1600646399999,60928.1154,387,7266.20000000,20715.0029
3,1600646400000,2.9110,3.0337,2.5500,2.7795,19867.49000000,1600732799999,55417.0202,345,10019.54000000,28579.9558
4,1600732800000,2.7378,3.0089,2.6593,2.9629,58835.67000000,1600819199999,168942.4414,435,12448.52000000,35901.7861
...,...,...,...,...,...,...,...,...,...,...,...
585,1650931200000,100.9758,102.0976,94.6210,95.7159,74880.17000000,1651017599999,7365647.2794,21625,38473.68000000,3785026.9001
586,1651017600000,95.7381,100.7525,94.9235,98.3570,65655.07000000,1651103999999,6438104.7141,21021,33831.09000000,3317102.8450
587,1651104000000,98.3136,100.5527,96.3708,98.2014,68097.56000000,1651190399999,6712283.8881,19725,33666.71000000,3319913.7655
588,1651190400000,98.2356,98.7000,92.3313,93.9566,73629.15000000,1651276799999,7004742.3399,17715,37734.88000000,3589963.9147


In [32]:
klines_df = klines_df.drop(columns='NaN')

In [33]:
klines_df_close = klines_df['close']
klines_df_close = klines_df_close.astype('float')

In [34]:
klines_df['close'] = klines_df['close'].str.replace('','')
klines_df['close'] = klines_df['close'].str.replace('','')
klines_df['close'] = klines_df['close'].astype('float')
klines_df.dtypes

open time                         int64
open                             object
high                             object
low                              object
close                           float64
volume                           object
close time                        int64
quote asset volume               object
number of trades                  int64
taker buy base asset volume      object
taker buy quote asset volume     object
dtype: object

In [35]:
window_length = 14
# Initialize containers for avg. gains and losses

gains = []
losses = []
# Create a container for current lookback prices
window = []
# Keeps track of previous average values
prev_avg_gain = None
prev_avg_loss = None
# Create a container for our final output (as a csv)
output = [['date', 'close', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rsi']]

In [36]:

for i, close in enumerate(klines_df_close):
    if i == 0:
        window.append(close)
        output.append([i+1, close, 0, 0, 0, 0, 0])
        continue
    # After the first period, calculate the difference
    # between price and previous price as a rounded value
    difference = round(klines_df_close[i] - klines_df_close[i - 1], 2)

    if difference > 0:
        gain = difference
        loss = 0
# Record negative differences as losses
    elif difference < 0:
        gain = 0
        loss = abs(difference)
# Record no movements as neutral
    else:
        gain = 0
        loss = 0
    gains.append(gain)
    losses.append(loss)

    if i < window_length:
        window.append(close)
        output.append([i+1, close, gain, loss, 0, 0, 0])
        continue

    if i == window_length:
        avg_gain = sum(gains) / len(gains)
        avg_loss = sum(losses) / len(losses)   

        prev_avg_gain = avg_gain
        prev_avg_loss = avg_loss

# Round for later comparison (optional)
        avg_gain = round(avg_gain, 2)
        avg_loss = round(avg_loss, 2)
        prev_avg_gain = round(prev_avg_gain, 2)
        prev_avg_loss = round(prev_avg_loss, 2)
   
        rs = round(avg_gain / avg_loss, 2)

        rsi = round(100 - (100 / (1 + rs)), 2)

                 

In [37]:
   
# Remove oldest values
window.append(close)
window.pop(0)
gains.pop(0)
losses.pop(0)
# Save Data
output.append([i+1, close, gain, loss, avg_gain, avg_loss, rsi])
    
     




In [39]:
output_df = pd.DataFrame(output)

In [41]:
RSI_SOL_output = output_df.to_csv('RSI_SOL_output.csv', index = True)
